In [1]:
import numpy as np
import cv2
import glob
import pickle


def populate_points(images,objpoints,objpt,imgpoints,nx,ny):
    for im in images:
        img = cv2.imread(im)
        imsize = img.shape[1::-1]
        #convert to gray
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray,(nx,ny),None)

        if ret == True:
            objpoints.append(objpt) #always append the same points
            imgpoints.append(corners) #append new corners

    return objpoints,imgpoints

def calibrate(imsize, objpoints, imgpoints):

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints,imsize,None,None)
    
    return ret, mtx, dist, rvecs, tvecs


def run_calibration_process():
    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')    

    #assuming all images have the same size
    img = cv2.imread(images[0])
    imsize = img.shape[1::-1]

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.    

    nx = 9
    ny = 6

    objpt = np.zeros((nx*ny,3),np.float32)
    #x,y coordinates, z remains zero
    #mgrid returns the coordinates values for a given gridsize
    #then we shape the coordinates back in two colums one for x and one for y
    objpt[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2) 

    #populate obj and img points 
    objpoints,imgpoints = populate_points(images,objpoints,objpt,imgpoints,nx,ny)
    #calibrate the camera
    ret, mtx, dist, rvecs, tvecs = calibrate(imsize, objpoints, imgpoints)
    print("camera calibration done")
    return mtx, dist, rvecs, tvecs



In [2]:
mtx, dist, rvecs, tvecs  = run_calibration_process()

camera_params = {'mtx':mtx,'dist':dist,'rvecs':rvecs,'tvecs':rvecs}

pickle.dump(camera_params, open( "camera_calibration.p", "wb" ) )



camera calibration done
